In [1]:
%matplotlib inline
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from data import *
from baseline_smc import *
from plots import *
from torch.distributions.dirichlet import Dirichlet
import sys
import time
import datetime
sys.path.append('/home/hao/Research/probtorch/')
import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.5.0a0+3bb8c5e cuda: True


In [2]:
T = 50
K = 3
D = 2

## Model Parameters
num_particles = 50
num_particles_smc = 30
NUM_HIDDEN = 32
NUM_LATENTS = K*K
NUM_OBS = 2 * D

NUM_EPOCHS = 100
LEARNING_RATE = 1e-3
CUDA = False
RESTORE = False
PATH_ENC = "hmm-smc-%dIS-enc-%s" % (num_particles, datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

In [3]:
Ys_array = np.load('sequences.npy')[:200]
As_true = torch.from_numpy(np.load('transitions.npy')).float()
Zs_true = torch.from_numpy(np.load('states.npy')).float()
mu_ks = torch.from_numpy(np.load('means.npy')).float()
cov_ks = torch.from_numpy(np.load('covariances.npy')).float()
Pi = torch.from_numpy(np.load('init.npy')).float()
alpha_trans_0 = initial_trans_prior(K)
Grad_Steps = Ys_array.shape[0]

In [4]:
class Encoder(nn.Module):
    def __init__(self, num_obs=NUM_OBS,
                       num_hidden=NUM_HIDDEN,
                       num_latents=NUM_LATENTS):
        super(self.__class__, self).__init__()
        self.enc_hidden = nn.Sequential(
            nn.Linear(num_obs, num_hidden),
            nn.Tanh())
        self.latent_dir = nn.Sequential(
            nn.Linear(num_hidden, num_latents))
        
    def forward(self, obs, prior):
        A_sample = torch.zeros((K, K))
        hidden = self.enc_hidden(obs)
        variational = F.softmax(self.latent_dir(hidden), -1).sum(0).view(K, K) + prior
        for k in range(K):
            A_sample[k] = Dirichlet(variational[k]).sample()
        return variational, A_sample

In [5]:
def initialize():
    enc = Encoder()
    if CUDA:
        enc.cuda()
    optimizer =  torch.optim.Adam(list(enc.parameters()),lr=LEARNING_RATE)    
    return enc, optimizer
enc, optimizer = initialize()

In [ ]:
KLs = []
EUBOs = []
ESSs = []
ELBOs = []
for epoch in range(NUM_EPOCHS):
    time_start = time.time()
    np.random.shuffle(Ys_array)
    Ys = torch.from_numpy(Ys_array).float()
    ave_elbo = 0.0
    ave_eubo = 0.0
    for n in range(Grad_Steps):
        optimizer.zero_grad()
        eubo, kl, ess, variational, elbo = oneshot_sampling(enc, alpha_trans_0, Pi, mu_ks, cov_ks, Ys[n], T, D, K, num_particles, num_particles_smc)
        ave_elbo += elbo
        ave_eubo += eubo
        eubo.backward()
        optimizer.step()
        
    EUBOs.append(ave_eubo / Grad_Steps)
    ELBOs.append(ave_elbo / Grad_Steps)
        
    time_end = time.time() 
    print('epoch : %d, eubo : %f, elbo : %f (%ds)' % (epoch, ave_eubo, ave_elbo, time_end - time_start))

if RESTORE:
    enc.load_state_dict(torch.load(PATH_ENC))
else:
    torch.save(enc.state_dict(), PATH_ENC)
    save_params(EUBOs, ELBOs, PATH_ENC)

epoch : 0, eubo : -42780.136719, elbo : -45807.843750 (861s)
epoch : 1, eubo : -42610.285156, elbo : -46189.593750 (860s)
epoch : 2, eubo : -42572.484375, elbo : -46112.382812 (841s)
epoch : 3, eubo : -42556.699219, elbo : -46079.984375 (839s)
epoch : 4, eubo : -42562.535156, elbo : -46032.488281 (839s)
epoch : 5, eubo : -42563.218750, elbo : -46069.605469 (842s)
epoch : 6, eubo : -42512.800781, elbo : -46416.437500 (842s)
epoch : 7, eubo : -42539.101562, elbo : -46120.394531 (843s)
epoch : 8, eubo : -42514.054688, elbo : -46259.667969 (838s)
epoch : 9, eubo : -42583.367188, elbo : -45801.718750 (832s)
epoch : 10, eubo : -42506.628906, elbo : -45881.664062 (840s)
epoch : 11, eubo : -42507.261719, elbo : -46109.273438 (846s)
epoch : 12, eubo : -42507.707031, elbo : -46060.167969 (845s)
epoch : 13, eubo : -42492.980469, elbo : -46012.851562 (864s)
epoch : 14, eubo : -42513.484375, elbo : -45877.625000 (854s)
epoch : 15, eubo : -42485.421875, elbo : -46013.500000 (900s)
epoch : 16, eubo :

In [ ]:
plot_dirs(latents_dirs, alpha_trans_0, Z_true, T, K, vmax=15)

In [ ]:
def plot_results(EUBOs, ELBOs, KLs, ESSs, PATH_ENC):
    fig, ax = plt.subplots(figsize=(16, 24))
    ax1 = fig.add_subplot(3,1,1)
    x = np.arange(len(EUBOs))
    ax1.plot(EUBOs, 'r-', label='eubo')
    ax1.plot(ELBOs, 'b-', label='elbo')
    ax1.legend()
    ax1.set_xlabel('epochs')
    ax2 = fig.add_subplot(3,1,2)
    ax2.plot(KLs, 'g-', label='KL')
    ax3 = fig.add_subplot(3, 1, 3)
    ax3.plot(np.array(ESSs))
    fig.savefig(PATH_ENC + 'results.png')
plot_results(EUBOs, ELBOs, KLs, ESSs, PATH_ENC)

In [ ]:


Zs, log_weights, log_normalizer = smc_hmm(Pi, A_true, mu_ks, cov_ks, Y, T, D, K, num_particles_smc)
Z_true = resampling_smc(Zs, log_weights)

In [ ]:
accuracy = []
for i in range(1000):
    alpha_trans_0 = initial_trans_prior(K)
    init_v = init_velocity(dt)
    T = np.random.randint(T_min, T_max)
    STATE, mu_ks, cov_ks, Pi, Y, A_true, Zs_true = generate_seq(T, K, dt, Boundary, init_v, noise_cov)

    Y_pairwise = torch.cat((Y[:-1].unsqueeze(0), Y[1:].unsqueeze(0)), 0).transpose(0, 1).contiguous().view(T-1, 2*D)
    variational, A_sample = enc(Y_pairwise, alpha_trans_0)
    Zs, log_weights, log_normalizer = smc_hmm(Pi, A_sample, mu_ks, cov_ks, Y, T, D, K, num_particles_smc)
    Z_ret = resampling_smc(Zs, log_weights)

    accuracy.append((Zs_true.nonzero()[:, 1] == Z_ret.nonzero()[:, 1]).sum().float().item() / T)

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax1 = fig.add_subplot(1,1,1)
ax1.plot(accuracy, 'ro')
ax1.set_ylim(0,1)
fig.savefig('accuracy_smc_noisy.png')

In [ ]:
torch.Tensor([3]) == torch.Tensor([3])

In [ ]:
accuracy